# STA 141B Assignment 3

Due __Nov 10, 2023__ by __11:59pm__. Submit your work by uploading it to Gradescope through Canvas.

Please rename this file as __"LastName_FirstName_hw3"__ and export it as as pdf-file. 

The objective of this assignment is acquire data via web APIs.  

Instructions:

1. Provide your solutions in new cells following each exercise description. Create as many new cells as necessary. Use code cells for your Python scripts and Markdown cells for explanatory text or answers to non-coding questions. Answer all textual questions in complete sentences.

2. Prioritize code readability. Just as in writing a book, the clarity of each line matters. Adopt the __one-statement-per-line__ rule. If you have a lengthy code statement, consider breaking it into multiple lines for clarity. (Please note: violating the one-statement-per-line rule will result in a one-point deduction for each offending line.)

3. To help understand and maintain code, you should always add comments to explain your code. Use the hash symbol (#) to start writing a comment (homework without any comments will automatically receive 0 points).

4. Submit your final work as a __.pdf__ file on __Gradescope__. To convert your .ipynb file into one of these formats, navigate to "File", select "Download as", and then choose either "PDF via LaTeX" or "HTML". If "PDF via LaTeX" does not work for you, export to "HTML", and then use Chrome to print the .html file into PDF. Gradescope only accepts PDF files.

5. This assignment will be graded on your proficiency in programming. Be sure to demonstrate your abilities and submit your own, correct and readable solutions. 

### Problem 1: Delphi’s Epidata API [2 Points]

In order to query a public API, it is important to carefully read the documentation. Consider Delphi’s Epidata API, which - amongst others - tracks and provides data about the Covid-19 disease. We are interested in querying from the COVIDcast Epidata API. Query the API directly, do not use the `covidcast` module!

#### Exercise

a) Make three requests, retrieving the signal `smoothed_wcli` and `smoothed_whh_cmnty_cli` from the `fb-survey` datasource as well as the `confirmed_7dav_incidence_prop` signal from the `jhu-csse` on `2022-01-01` for all states. Transform the response to `pandas.DataFrame`. use `conf_jhu.info()` to check the data is correct.

b) Next, create a new table with three columns with the signals `smoothed_wcli`, `smoothed_whh_cmnty_cli` and `confirmed_7dav_incidence_prop`. Drop missing values and compute the sample correlation matrix. Show the sample correlation matrix.

In [3]:
# a)

import requests
import pandas as pd

def fetch(data_source, signal):
    #base url as well as the object to hold our parameters
    datasource_url = 'https://api.delphi.cmu.edu/epidata/covidcast/'
    our_params = {
        'data_source' : data_source,
        'signal' : signal,
        'time_type': 'day',
        'geo_type' : 'state',
        'time_values': '20220101',
        'geo_value': '*'
    }
    
    response = requests.get(datasource_url, params = our_params)
    #if receive a positive response, we return a populated df, else return empy one
    if response.status_code == 200:
        data = response.json()
        return pd.DataFrame(data['epidata'])
    else:
        print('there was an errror fetching your data')
        return pd.DataFrame()
    
    
# Your code here 
scl_fb = fetch('fb-survey', 'smoothed_wcli')
shccl_fb = fetch('fb-survey', 'smoothed_whh_cmnty_cli')
conf_jhu = fetch('jhu-csse', 'confirmed_7dav_incidence_prop')

conf_jhu.info()

# b)
#merge smoothed_wcli and smoother_whh as well as a second merge with the rest of the data
concat_df =  pd.merge(scl_fb[['geo_value', 'value']], 
                      shccl_fb[['geo_value', 'value']], 
                      on = 'geo_value', how = 'outer')

combined_df = pd.merge(concat_df, conf_jhu[['geo_value','value']], 
                       on = 'geo_value', how = 'outer')

#drop any null values from the final df
combined_df.dropna()

final_correlation_matrix = combined_df.corr()

print(final_correlation_matrix)
# Your code here

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   geo_value            56 non-null     object 
 1   signal               56 non-null     object 
 2   source               56 non-null     object 
 3   geo_type             56 non-null     object 
 4   time_type            56 non-null     object 
 5   time_value           56 non-null     int64  
 6   direction            0 non-null      object 
 7   issue                56 non-null     int64  
 8   lag                  56 non-null     int64  
 9   missing_value        56 non-null     int64  
 10  missing_stderr       56 non-null     int64  
 11  missing_sample_size  56 non-null     int64  
 12  value                56 non-null     float64
 13  stderr               0 non-null      object 
 14  sample_size          0 non-null      object 
dtypes: float64(1), int64(6), object(8)
memory 

### Problem 2: University of California Employee Pay [6 Points]

As a public organization, the compensations of employees of all institutions of the University of California are freely accessible. These reports cover UC's career faculty and staff employees, as well as part-time, temporary and student employees. It is accessible [here](https://ucannualwage.ucop.edu). Internally, the data requested by the search mask is queried using an undocumented API. We are interested to compare the gross pay to the number of units actually taught. To this end, we use the [UCD registry](https://registrar-apps.ucdavis.edu/courses/search/index.cfm). 

__Hint__: The University of California Employee Pay API returns an object that is not in json format, since it includes single instead of double quotation marks. Instead of the method `.json`, the data can be accessed using the function `json.loads` from the `json` module, after replacing the quotation marks: 

    result = requests. ... # this is the request
    import json
    json.loads(result.text.replace("\'", "\"")) # this returns a dict
    
Also, throughout this exercise, we will extracts digits from strings. This can be achieved using the `re` package: 

    import re
    re.findall('\d+', "this is a test 12 string with some 23, 0 digits in it!") # returns ['12', '23', '0']    

#### Exercise

a) Query the wage API to obtain the annual wage data of 2021 of all UC Davis professors. Remove all privaticed names and return a `pandas.DataFrame` with the first name, last name and gross pay, sorted in descending order. Print the first __six__ rows. The first three are: 

```
> wage_table.head(3)
	    FirstName	  LastName	Pay
1124	GRIFFITH	   HARSH	    1197167.0
1503	CHRISTOPHER	KREULEN	  1174990.0
1136	BRIAN	      HAUS	     926743.0

```

b) Next, find out how many _units_ each person of this data frame has taught. To reduce effort, only consider the first 100 rows of `wage_table`, but make your code scalable by using `threading`. Query these employees in the [UCD registry](https://registrar-apps.ucdavis.edu/courses/search/index.cfm). If courses were taught, those will be displayed here. For example `KRAMLINGER` and `Fall Quarter 2023` gives six courses. Each has a link with additional information (e.g., `view 50589`). From there, additional information can be accessed. For the instructional purposes, we will retrieve the value of __Units__ from here, for all courses taught. 

Aggregate the units for all courses taught in all terms in 2021 for the 100 largest entries from a). Create a new data set, `full_table`, that contains these 100 largest entries from a) and the total units. Add another column that calculates the pay per unit and re-sort the vector. Print the first __six__ rows.

```
> full_table.head(3) 
	   FirstName  LastName	Pay	     Units	PayPerUnit
1749   RICHARD	MARDER	  552965.0	3	    184321.666667
299	RICHARD	BOLD	    532720.0	5	    106544.000000
1270   SAMUEL	 HWANG  	 732169.0	8	    91521.125000
```


In [4]:
# a) 
import json
import pandas as pd
import requests
import time

#base url parameter to insert into object
base_url = "https://ucannualwage.ucop.edu/wage/search.action"
prof_data = []    

response = requests.get(base_url, params={
        "year": '2021',
        "location": "Davis",
        "title": "PROF",
        "sidx": "EAW_LST_NAM",
        "Page": 1,
        "rows":3200
    })

# turn parsed data into a proper format
parsed_result = json.loads(response.text.replace("'", "\""))

# extracting data from the parsed_result dictionary, with the key 'rows'
prof_rows_data = parsed_result['rows']
prof_data.extend(prof_rows_data)

#initiate empty prof list to append to
prof_list = []
#loop to populate the empty array
for professor in prof_data:
    professor_dict = {
        'Year': professor['cell'][1],
        'First_Name': professor['cell'][3],
        'Last_Name': professor['cell'][4],
        'Pay': float(professor['cell'][6])
    }
    prof_list.append(professor_dict)

#arrange the table via pay column values    
wage_table = pd.DataFrame(prof_list)
wage_table['Pay'] = wage_table['Pay'].astype(float)

# now get descending order of total pay
wage_table = wage_table.sort_values(by='Pay', ascending=False)

#print statement
print(wage_table.head(n=6))

      Year        First_Name Last_Name        Pay
2008  2021          GRIFFITH     HARSH  1197167.0
1629  2021       CHRISTOPHER   KREULEN  1174990.0
1996  2021             BRIAN      HAUS   926743.0
2325  2021       CHRISTOPHER     EVANS   912962.0
871   2021            ROBERT   RANDALL   909825.0
811   2021  ROLANDO FIGUEROA   ROBERTO   901841.0


In [ ]:
# b) 

import re
import lxml.html as lx

# Your code here

import concurrent.futures, threading

# Your code here

#full_table.head(n = 6)